# Strade Urbane


In [21]:
from shapely import MultiLineString, LineString, Polygon, MultiPolygon

import geopandas as gpd

In [22]:
gdf = gpd.read_file(PATH_STRADE_URBANE_RAW)
gdf = gdf.set_crs("EPSG:7794")
gdf = gdf.to_crs("EPSG:4326")

In [23]:
def polygon_to_multilinestring(geom):
    if isinstance(geom, Polygon):
        # Converte esterno e interni in LineString e li unisce
        lines = [LineString(geom.exterior.coords)] + [LineString(ring.coords) for ring in geom.interiors]
        return MultiLineString(lines)
    elif isinstance(geom, MultiPolygon):
        # Applica la conversione a ogni poligono
        lines = []
        for poly in geom.geoms:
            lines.append(LineString(poly.exterior.coords))
            lines.extend([LineString(ring.coords) for ring in poly.interiors])
        return MultiLineString(lines)
    else:
        return geom  # Lascia intatto se è già una LineString o MultiLineString

gdf["geometry"] = gdf["geometry"].apply(polygon_to_multilinestring)

In [24]:
gdf.to_file(PATH_STRADE_URBANE_CLEAN, driver="GeoJSON")